In [9]:
!pip install ur-rtde==1.4.9

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\linhe\\AppData\\Local\\Temp\\pip-uninstall-wx9r881n\\dashboard_client.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.




     ---------------------------------------- 2.1/2.1 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: ur-rtde
    Found existing installation: ur-rtde 1.5.1
    Uninstalling ur-rtde-1.5.1:
      Successfully uninstalled ur-rtde-1.5.1


In [1]:
import socket
import struct
import time 

class UrIO:
    def __init__(self, ip, port=30001):
        self._socket = None
        self._ip = ip
        self._port = port
    
    def connect(self):
        TIMEOUT = 1 # seconds
        print ("Attempting to connect to %s on port %s" % (self._ip, self._port))
        try:
            self._socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self._socket.settimeout(TIMEOUT)
            self._socket.connect((self._ip, self._port))
            print ("Connected to %s on port %s" % (self._ip, self._port))
        except socket.error as error:
            print("Connection to %s on port %s failed: %s" % (self._ip, self._port, error))
        
    def send_URScript(self, msg, recno = 1116):
        if not self._socket is None:
            try:
                self._socket.send(msg)
                data = self._socket.recv(recno)
                return True, data
            except:
                print("Error sending.. \n")
                return False, "Error sending data"
    
    def set_digital_out(self, n, state) -> bool:
        if (n >= 0 and n <= 9) and (state in [True, False]):
            msgs = f'sec set_std_DOUT():\n\tset_digital_out({n}, {state})\nend\n'
            msgb = msgs.encode()
            res, data = self.send_URScript(msgb)
            # print(f"res:{res} data:{data}")
            return res


In [1]:
import numpy as np
from dashboard_client import DashboardClient
from rtde_io import RTDEIOInterface as RTDEIO
from rtde_receive import RTDEReceiveInterface as RTDEReceive
from rtde_control import RTDEControlInterface as RTDEControl

ip = "192.168.1.30"


In [3]:
class URRobot():
    def __init__(self):
        self.rtde_db = None
        self.rtde_rec = None
        self.rtde_ctrl = None
        self.rtde_io = None
        
    def connect(self):
        db_status = None
        rec_status = None
        ctrl_status = None
        io_status = None
        db_status = self.connect_db()
        if db_status:
            rec_status = self.connect_rec()
            if rec_status:
                ctrl_status = self.connect_ctrl()
                # if ctrl_status:
                #     io_status = self.connect_io()
        print(f"db: {db_status} rec: {rec_status} ctrl: {ctrl_status} io: ")
            
    def connect_db(self):
        self.rtde_db = DashboardClient(ip)
        try:
            self.rtde_db.connect()
        except:
            return False
        if self.rtde_db.isConnected():
            print("db is connected.")
            return True
        else:
            print("db is not connected.")
            return False
    
    def connect_rec(self):
        try:
            self.rtde_rec = RTDEReceive(ip)
            return True
        except Exception as e:
            print(f"fail {e}")
            return False

    
    def connect_ctrl(self):
        try:
            RTDEControl(ip).disconnect()
            self.rtde_ctrl = RTDEControl(ip)
            return True
        except:
            print(f"fail ctrl")
            return False

        
    def connect_io(self):
        try:
            print("connect2io")
            self.rtde_io = UrIO(ip)
            self.rtde_io.connect()
            return True
        except Exception as e:
            print(f"fail {e}")


    def get_status(self):
        dic_robot_status = {
            0: "Is power on",
            1: "Is program running",
            2: "Is teach button pressed",
            3: "Is power button pressed",
            7: "Freedrive"
        }
        robot_status = self.rtde_rec.getRobotStatus()
        dic_safety_bit = {
            0: "Is normal mode",
            1: "Is reduced mode",
            2: "Is protective stopped",
            3: "Is recovery mode",
            4: "Is safeguard stopped",
            5: "Is system emergency stopped",
            6: "Is robot emergency stopped",
            7: "Is emergency stopped",
            8: "Is violation",
            9: "Is fault",
            10: "Is stopped due to safety",
            1216: "Emergency",
        }
        safety_status_bit = self.rtde_rec.getSafetyStatusBits()
        dic_mode_bit={
            -1: "ROBOT_MODE_NO_CONTROLLER",
            0: "ROBOT_MODE_DISCONNECTED",
            1: "ROBOT_MODE_CONFIRM_SAFETY",
            2: "ROBOT_MODE_BOOTING",
            3: "ROBOT_MODE_POWER_OFF",
            4: "ROBOT_MODE_POWER_ON",
            5: "ROBOT_MODE_IDLE",
            6: "ROBOT_MODE_BACKDRIVE",
            7: "ROBOT_MODE_RUNNING",
            8: "ROBOT_MODE_UPDATING_FIRMWAR",
        }
        robot_mode_bit = self.rtde_rec.getRobotMode()
        return [robot_status, safety_status_bit, robot_mode_bit]

In [4]:
a = URRobot()

# DB

In [74]:
a.connect_db()

False

In [52]:
a.rtde_db.unlockProtectiveStop()

In [8]:
try:
    a.rtde_db.closeSafetyPopup()
    a.rtde_db.restartSafety()
    print("done")
except:
    print("reconnect db")

done


In [20]:
try:
    a.rtde_db.powerOn()
    a.rtde_db.brakeRelease()
    print("done")
except:
    print("reconnect db")

reconnect db


# RECEIVE

In [14]:
a.connect_rec()

True

In [61]:
a.rtde_rec.getActualQ()

[-1.55931264558901,
 -1.1305913788131257,
 -2.6331331729888916,
 -0.1187775892070313,
 -1.609955135975973,
 0.8753644824028015]

In [15]:
a.get_status()

[0, 1536, 3]